# **Sesión 3:** Procesamiento de imagenes y extracción de características

## **Librerías**

In [7]:
import cv2
print("OpenCV should be 4.8.0.76 Current version:", cv2.__version__)
from typing import List
import numpy as np
import imageio
import copy
import glob
import matplotlib.pyplot as plt
from typing import Optional
from utils import *
import math
import os

OpenCV should be 4.8.0.76 Current version: 4.8.0


## **Apartado A:** Filtro Gaussiano y Detección de bordes: Sobel y Canny

El objetivo de este apartado es detectar los bordes de las imágenes de la carpeta ``data/partA-B``. Para ello, deberá seguir los siguientes pasos:

1. **Tarea A.1.** Defina el método ``gaussian_blur()`` que aplique un filtro gausiano para obtener imágenes borrosas. Siga todas las indicaciones del enunciado.
2. **Tarea A.2.** Aplique el método ``gaussian_blur()`` a todas las imágenes en ``data/partA-B``.


3. **Tarea A.3.** Defina la función ``sobel_edge_detector()`` que detecte bordes con el método Sobel. Siga todas las indicaciones del enunciado.
4. **Tarea A.4.** Aplique el método ``sobel_edge_detector()`` a todas las imágenes en ``data/partA-B``.


5. **Tarea A.5.** Defina la función ``canny_edge_detector()`` que detecte bordes con el método Canny. Siga todas las indicaciones del enunciado.
6. **Tarea A.6.** Aplique el método ``canny_edge_detector()`` a todas las imágenes en ``data/partA-B``.

### **Tarea A.1:** Defina el método ``gaussian_blur()`` que aplique un filtro gausiano para obtener imágenes borrosas.

In [ ]:
# TODO Define the method
def gaussian_blur(img: np.array, sigma: float, filter_shape: Optional[List] = None, verbose: bool = False) -> np.array:
    # If not given, compute the filter shape 
    if filter_shape == None:
        k = max(1, math.ceil(4*sigma)) # semilado del filtro (al menos 1)
        filter_l = 2*k + 1  # tamaño siempre impar
        kh, kw = filter_l, filter_l
        # El rango de sigma es [-4*sigma, 4*sigma]. Se multiplica por dos para aplicarlo en ambas direcciones (x,y)
        # Le sumamos 1 para que sea impar y tenga un centro
    else:
        # Si se pasa manualmente, nos aseguramos de que las dimensiones sean impares
        kh, kw = int(filter_shape[0]), int(filter_shape[1])
        if kh % 2 == 0:
            kh += 1
        if kw % 2 == 0:
            kw += 1
    
    # Create the filter coordinates matrices
    y, x = np.mgrid[-kh//2 : kh//2+1 , -kw//2 : kw//2+1]

    # Ej si kh = 5, kw = 5, entonces queremos los rangos [-2, -1, 0, 1, 2] tanto en x como en y. 
    # Por eso se pone +1 en el final del mgrid

    # Define the formula that goberns the filter
    formula = np.exp(-(x**2 + y**2)/(2*sigma**2)).astype(np.float32) # Ponemos tipo np.float32 para garantizar compatibilidad y precisión con OpenCV, asegurando que el kernel Gaussiano tenga el tipo de dato correcto
    gaussian_filter = formula / formula.sum() # Normalizamos el filtro para que la suma de todos sus valores sea 1 y se mantenga el brillo de la imagen
    
    # Process the image
    gb_img = cv2.filter2D(img, -1, gaussian_filter)
    
    if verbose:
        cv2.show_image(img=gb_img, img_name=f"Gaussian Blur: Sigma = {sigma}")
    
    return gaussian_filter, gb_img.astype(np.uint8) # Ponemos np.uint8 para guardar o visualizar el resultado en el mismo rango y tipo que imagen original

### **Tarea A.2.** Aplique el método ``gaussian_blur()`` a todas las imágenes en ``data``.

In [8]:
# TODO Get the gaussian blurred images using a list comprehension

# Definimos la función para cargar las imágenes
def load_images(filenames: List[str]) -> List[np.ndarray]:
    return [imageio.imread(filename) for filename in filenames]

# 1. Cargamos la imagenes
image_paths_AB = sorted(glob.glob("../data/partA-B/*.jpg"))
imgs_AB = load_images(image_paths_AB)

# 2. Definimos el valor de sigma
gauss_sigma = 2.0
gb_imgs = [gaussian_blur(img, sigma=gauss_sigma, filter_shape=None, verbose=False)[1] for img in imgs_AB]

# TODO Show images and save when needed
def show_image(img, img_name="imagen"):
    cv2.imshow(img_name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

def write_image(img, output_folder, filename):
    os.makedirs(output_folder, exist_ok=True) 
    img_path = os.path.join(output_folder, filename)
    cv2.imwrite(img_path, img)

# 4. Definimos la carpeta de salida
output_folder = "../output_blurred"

# 5. Mostramos y guardamos las imágenes
for i, (orig, gb_img) in enumerate(zip(imgs_AB, gb_imgs)):
    show_image(orig, f"original_{i}")
    show_image(gb_img, f"blurred_{i}_sigma{gauss_sigma}")
    
    write_image(gb_img, output_folder, f"blurred_{i}_sigma{gauss_sigma}.jpg")

C:\Users\crist\AppData\Local\Temp\ipykernel_16540\3136817771.py:5: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  return [imageio.imread(filename) for filename in filenames]


### **Tarea A.3:** Defina la función ``sobel_edge_detector()`` que detecte bordes con el método Sobel.

In [ ]:
# TODO Define the method


def sobel_edge_detector(img: np.array, filter: np.array, gauss_sigma: float, gauss_filter_shape: Optional[List] = None, verbose: bool = False) -> np.array:
    # TODO Transform the img to grayscale
    gray_img = cv2.cvtColor()
    
    # TODO Get a blurry img to improve edge detections
    blurred = gaussian_blur(img=None, sigma=None, filter_shape=None, verbose=verbose)
    
    # Re-scale
    blurred = blurred/255
    
    # TODO Get vertical edges
    v_edges = cv2.filter2D()
    
    # TODO Transform the filter to get the orthogonal edges
    filter = filter
    
    # TODO Get horizontal edges
    h_edges = cv2.filter2D()
    
    # TODO Get edges
    sobel_edges_img = np.hypot()
    
    # Get edges angle
    theta = np.arctan2(None, None)
    
    # Visualize if needed
    if verbose:
        show_image(img=sobel_edges_img, img_name="Sobel Edges")
    
    return np.squeeze(sobel_edges_img), np.squeeze(theta)

### **Tarea A.4.** Aplique el método ``sobel_edge_detector()`` a todas las imágenes en ``data``.

In [ ]:
# TODO Define a sigma value
gauss_sigma = None

# TODO Define the Sobel filter
sobel_filter = np.array()

# TODO Get the edges detected by Sobel using a list comprehension
sobel_edges_imgs = []

### **Tarea A.5:** Defina la función ``canny_edge_detector()`` que detecte bordes con el método Canny.

In [ ]:
# TODO Define the method
def canny_edge_detector(img: np.array, sobel_filter: np.array, gauss_sigma: float, gauss_filter_shape: Optional[List] = None, verbose: bool = False):
    # TODO Call the method sobel_edge_detector()
    sobel_edges_img, theta = sobel_edge_detector()
    
    # TODO Use NMS to refine edges
    canny_edges_img = non_max_suppression()
    
    if verbose:
        show_image(canny_edges_img, img_name="Canny Edges")
        
    return canny_edges_img

### **Tarea A.6.** Aplique el método ``canny_edge_detector()`` a todas las imágenes en ``data``.

In [ ]:
# TODO Define Sobel filter
sobel_filter = None

# TODO Define a sigma value for Gauss
gauss_sigma = None

# TODO Define a Gauss filter shape
gauss_filter_shape = [None, None]

# TODO Get the edges detected by Canny using a list comprehension
canny_edges = []

### **Pregunta A.1:** Añada ruido a las imágenes de la carpeta ``data``. Compare los resultados que obtiene al aplicar su filtro Sobel con y sin filtro Gausiano.

In [ ]:
# TODO Homework

### **Pregunta A.2:** Utilice la librería ``scikit-image`` y compare el efecto de los filtros Sobel, Canny y Prewitt sobre las imágenes de la carpeta ``data``. ¿Qué diferencias observa entre los filtros? ¿Puede obtener alguna conclusión y/o patrón?

In [ ]:
# TODO Homework

## **Apartado B:** Operadores Morfológicos

Para resolver este partado, deberá seguir los siguientes pasos:

1. **Tarea B.1.** Defina el método ``binarize()`` para binarizar imágenes.
2. **Tarea B.2.** Defina el método ``custom_dilate()``.
3. **Tarea B.3.** Defina el método ``custom_erode()``.
4. **Pregunta B.1** Aplique los métodos ``custom_dilate()`` y ``custom_erode()`` a todas las imágenes de la carpeta ``data``.


### **Tarea B.1.** Defina el método ``binarize()`` para binarizar imágenes.

In [ ]:
# TODO Homework: define the binarization method
def binarize(img: np.array, threshold: int = 127):
    binary_img = None
    return binary_img

### **Tarea B.2.** Defina el método ``custom_dilate()``

In [ ]:
# TODO Homework: define the dilation method
def custom_dilate(img):
    # TODO pad the original image so it can keep dimensions after processing
    padded = np.pad()
    
    # TODO get img shape
    width = None
    height = None
    
    # TODO Create an element with the same dimensions as the padded img
    dilated = np.zeros()
    
    for j in range(height):
        for i in range(width):
            # TODO Add logic to the operation
            pass
            
    # TODO Select the region of interest (ROI). Modify if needed
    dilated = dilated[1:height+1, 1:width+1]
    
    return dilated

### **Tarea B.3.** Defina el método ``custom_erode()``

In [ ]:
# TODO Homework: define the erotion method
def custom_erode(img):
    eroded = None
    
    return eroded

### **Pregunta B.1** Aplique los métodos ``custom_dilate()`` y ``custom_erode()`` a todas las imágenes de la carpeta ``data``.

In [ ]:
# TODO Homework